In [1]:
%pip install scikit-learn
%pip install imutils
%pip install opencv-python-headless
%pip install numpy
%pip install argparse
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.
  Preparing metadata (setup.py) ... done
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25836 sha256=348e28fea74293469a07031c308bb00453731148314aab8e1bfdcdc44cf7ab0a
  Stored in directory: /home/topaci/.cache/pip/wheels/85/cf/3a/e265e975a1e7c7e54eb3692d6aa4e2e7d6a3945d29da46f2d7
Successfully built imutils
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 31.8 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from imutils import paths
import numpy as np
import argparse
import imutils
import cv2
import os
import _pickle as cPickle
from matplotlib import pyplot as plt

In [3]:
# import cv2
# import numpy as np
# import os
import sys
import time
from tkinter import Tk
from tkinter import filedialog
from tkinter import Tcl

def get_file_list(file_extension, folder_path):
    in_folder_path = folder_path

    files = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(in_folder_path):
        for file in f: 
            if file_extension in file:
                files.append(os.path.join(r, file))

    # print(len(files))
    return Tcl().call('lsort', '-dict', files)

def get_file_name_and_ext(file_path):
    file_name, file_ext = os.path.splitext(os.path.basename(file_path))
    return file_name, file_ext

def extract_files_names(file_paths):
    files_names = []
    for img_path in file_paths:
        image_name, img_ext = get_file_name_and_ext(img_path)
        # if (image_name.startswith('img')):
        files_names.append(image_name)
    return files_names

In [4]:
def digit_to_label(digit):
  label = int(digit)
  if label == 0:
      label = 'overripe'
  elif label == 1:
      label = 'ripe'
  elif label == 2:
      label = 'semi_ripe'
  elif label == 3:
      label = 'unripe'
  return label

In [6]:
def extract_color_histogram(image, bins=(8, 8, 8)):
  hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
  hist = cv2.calcHist([hsv], [0, 1, 2], None, bins, [0, 180, 0, 256, 0, 256])
  
  if imutils.is_cv2():
    hist = cv2.normalize(hist)
  else:
    cv2.normalize(hist, hist)
  
  return hist.flatten()

In [7]:
# image_names = extract_files_names(img_files)
# print(f"Name of the first image file: {image_names[img_id]}\nNumber of the extracted image names: {len(image_names)}\n")

# img_label = image_names[img_id].split('_')[1]
# print(f"Label of the image: {img_label}\n")
total_labels = []

def get_image_data(dir_path, file_ext=".png"):
    data = []
    labels = []

    img_files = get_file_list(file_ext, dir_path)
    img_id = 0
    print(f"The full path of the first image file: {img_files[img_id]}\nNumber of image files in the selected folder: {len(img_files)}\n")
    start_time = time.time()
    for i, img_path in enumerate(img_files):
      image_name, image_ext = get_file_name_and_ext(img_path)
      img_label = image_name.split('_')[0]
      img_label = digit_to_label(img_label)
      # image = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
      image = cv2.imread(img_path)


      if (image is not None):
        hist = extract_color_histogram(image)
        data.append(hist)
        labels.append(img_label)
        total_labels.append(img_label)
      else : print(imagePath); print(image)


      if i > 0 and i % 1000 == 0:
          print("{}/{}".format(i, len(img_files)))

      # X.append(resized_image)
      # Y.append(img_label)
    end_time = time.time()
    print(f"{len(img_files)} image files have been loaded in {round((end_time - start_time), 2)} seconds.\n")
    return np.array(data), labels


In [8]:
file_ext = ".png"
dir_train_path = "../dataset/ivyqo/crop/train"
dir_test_path = "../dataset/ivyqo/crop/test"
train_data, train_label = get_image_data(dir_train_path)
test_data, test_label = get_image_data(dir_test_path)


The full path of the first image file: ../dataset/ivyqo/crop/train/0_0a1fd67636b678238bd91de546914006_0.png
Number of image files in the selected folder: 17741

1000/17741
2000/17741
3000/17741
4000/17741
5000/17741
6000/17741
7000/17741
8000/17741
9000/17741
10000/17741
11000/17741
12000/17741
13000/17741
14000/17741
15000/17741
16000/17741
17000/17741
17741 image files have been loaded in 8.83 seconds.

The full path of the first image file: ../dataset/ivyqo/crop/test/0_0a095606dd17e4f89874da3b29a518ef_0.png
Number of image files in the selected folder: 4078

1000/4078
2000/4078
3000/4078
4000/4078
4078 image files have been loaded in 1.77 seconds.



In [9]:
le = LabelEncoder()
fit_labels = le.fit_transform(total_labels)
print(fit_labels)
print(le.classes_)

[0 0 0 ... 3 3 3]
['overripe' 'ripe' 'semi_ripe' 'unripe']


In [10]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

names = [
          "K Nearest Neighbors",
          "Linear SVM",
          "RBF SVM",
          "Gaussian Process",
          "Decision Tree",
          "Random Forest",
          "Neural Net",
          "AdaBoost",
          "Naive Bayes",
          "QDA"
          ]

classifiers = [
                KNeighborsClassifier(3),
                SVC(kernel="linear", C=0.025),
                SVC(gamma=2, C=1),
                GaussianProcessClassifier(1.0 * RBF(1.0)),
                DecisionTreeClassifier(max_depth=5),
                RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
                MLPClassifier(alpha=1, max_iter=1000),
                AdaBoostClassifier(),
                GaussianNB(),
                QuadraticDiscriminantAnalysis()
                ]

In [11]:
classifier_id = 3
model = classifiers[classifier_id]
start_timeTr = time.time()
model.fit(train_data, train_label)
end_timeTr = time.time()
print(f"Training time: {round((end_time-start_time)/60,2)} minutes\n")

In [ ]:
for name, clf in zip(names, classifiers):
    print(f"Name of the Classifier: {name}")
    # model = classifiers[1]
    training_time_Start = time.time()
    clf.fit(train_data, train_label)
    training_time_End = time.time()
    print(f"Training time is : {round((training_time_End-training_time_Start)/60, 2)} minutes")
    predictions = clf.predict(test_data)
    
    print(classification_report(test_label, predictions, target_names=le.classes_))
    print("\n")


In [ ]:
image_path = img_files[13210]
print(image_path)

image_name, image_ext = get_file_name_and_ext(img_path)
img_label = image_name.split('_')[0]
img_label = digit_to_label(img_label)

singleImage = cv2.imread(image_path)
histt = extract_color_histogram(singleImage)
histt2 = histt.reshape(1, -1)
prediction = model.predict(histt2)
print(prediction)
print(f"Predicted as a {digit_to_label(prediction)}")
plt.imshow(cv2.cvtColor(singleImage, cv2.COLOR_BGR2RGB))
plt.show()